<a href="https://colab.research.google.com/github/MarioSigal/Aprendizaje-Automatico-I-y-II/blob/main/TP2/tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ebooklib beautifulsoup4 pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0:00:00


In [2]:
!pip install pdfplumber


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 65.1 MB/s eta 0:00:00


In [3]:
from ebooklib import epub
import ebooklib
from bs4 import BeautifulSoup
import pandas as pd
import re
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pdfplumber
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Funciones Auxiliares

In [5]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [6]:
def normalize_text_X(t):
    # Convertir a minúsculas y quitar puntuación
    t = t.lower()
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-záéíóúüñ0-9' -]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [7]:
def normalize_text_y(t):
    # Convertir a minúsculas y quitar puntuación
    t = re.sub(r'[\u200b-\u200f\uFEFF]', '', t)
    t = re.sub(r"[^a-zA-Záéíóúüñ0-9¿?,.' -ÁÉÍÓÚÜÑ]+", ' ', t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

In [8]:
def transformar_etiqueta(y, indice):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(indice)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append(1) #('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append(0) #("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append(3) #('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append(1) #('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append(2) #('","')
          else:
            puntuacion_finales.append(0) #("")

        else:
          puntuacion_finales.append(0) #("")

        if palabra.islower():
          capitalizaciones.append(0)

        elif palabra.istitle():
          capitalizaciones.append(1)

        elif palabra.isupper():
          capitalizaciones.append(3)

        else:
          capitalizaciones.append(2)


  etiquetas = np.column_stack([instancia_ids, token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [9]:
def convertir_epub_a_csv(archivo_epub='libro.epub'):
  # Cargar el libro
  book = ebooklib.epub.read_epub(archivo_epub)

  # Lista donde se guardarán los párrafos
  parrafos = []

  # Recorremos los ítems del libro
  for item in book.get_items():
      if item.get_type() == ebooklib.ITEM_DOCUMENT:
          # Parseamos el contenido HTML
          soup = BeautifulSoup(item.get_body_content(), 'html.parser')
          # Extraemos los párrafos
          for p in soup.find_all('p'):
            #print("p:",p, 'tipo: ', type(p))
            texto = p.get_text().strip()
            #print("TEXTO:",texto, ' tipo: ', type(texto))
            palabras = texto.split()
            #print("PALABRAS:",palabras, ' tipo: ', type(palabras))
            if len(palabras) < 20 or len(palabras) > 100:  # descartamos párrafos cortos
                continue
            if texto:
                parrafos.append(texto)

  df = pd.DataFrame({'parrafo': parrafos})
  df.to_csv("libro_parrafos.csv", index=False, encoding="utf-8")

  print(f"Se extrajeron {len(parrafos)} párrafos y se guardaron en 'libro_parrafos.csv'.")

In [12]:
def convertir_pdf_a_csv(pdf_path):
    """
    Convierte un archivo PDF en un CSV con una columna 'parrafo'.
    Divide el texto en los tipos de oraciónes que queremos apra entrenar
    """

    # Extraer texto del PDF
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            texto = pagina.extract_text()
            if texto:
                texto_total += " " + texto

    # Limpieza básica
    texto_total = re.sub(r"-\n", "", texto_total)   # une palabras cortadas
    texto_total = re.sub(r"\s+", " ", texto_total)  # espacios normales
    texto_total = texto_total.strip()

    # Dividir el texto en oraciones por ".", "?", "!"
    # Pero conservamos los signos al final
    oraciones = re.split(r'(?<=[\.\?\!])\s+', texto_total)
    oraciones = [o.strip() for o in oraciones if len(o.strip()) > 2]

    #Agrupar cada 1–3 oraciones en un bloque
    bloques = []
    buffer = []
    for o in oraciones:
        buffer.append(o)
        # si ya hay 3 oraciones, o la actual termina en "?" o "!"
        if len(buffer) >= 3 or re.search(r'[?!]$', o):
            bloques.append(" ".join(buffer))
            buffer = []

    # lo que quede
    if buffer:
        bloques.append(" ".join(buffer))

    # Filtrar solo las que contienen puntuación real
    bloques = [b.strip() for b in bloques if re.search(r'[.,¿?!]|[a-zA-ZáéíóúÁÉÍÓÚñ]$', b)]

    df = pd.DataFrame(bloques, columns=['parrafo'])
    df.to_csv('libro_parrafos.csv', index=False, encoding='utf-8-sig')
    return df

#_______________________________________________________________________________________________
# VERSIÓN 2 DE PDF-> CSV

def convertir_pdf_a_csv2(pdf_path):
    """
    Convierte un PDF en CSV pero corta los textos de forma aleatoria:
    - A mitad de una oración
    - Cada 1 oración
    - Cada 2 oraciones
    - Siempre corta al final de una pregunta
    Además filtra párrafos con muchos números.
    """

    # -------- Extraer texto --------
    texto_total = ""
    with pdfplumber.open(pdf_path) as pdf:
        for pagina in pdf.pages:
            txt = pagina.extract_text()
            if txt:
                texto_total += " " + txt

    texto_total = re.sub(r"-\n", "", texto_total)
    texto_total = re.sub(r"\s+", " ", texto_total).strip()

    # -------- Dividir en oraciones --------
    oraciones = re.split(r'(?<=[\.\?\!])\s+', texto_total)
    oraciones = [o.strip() for o in oraciones if len(o.strip()) > 2]

    bloques = []
    i = 0
    while i < len(oraciones):

        # si aparece una pregunta → cortar ahí
        if "¿" in oraciones[i] or "?" in oraciones[i]:
            bloques.append(oraciones[i])
            i += 1
            continue

        # decisión aleatoria
        modo = np.random.choice(["mitad", "1", "2"], p=[0.2, 0.4, 0.4])

        if modo == "mitad":
            # cortar a la mitad de la oración
            oracion = oraciones[i]
            palabras = oracion.split()
            mitad = max(3, len(palabras) // 2)
            bloque = " ".join(palabras[:mitad])  # mitad truncada
            bloques.append(bloque)
            i += 1

        elif modo == "1":
            bloques.append(oraciones[i])
            i += 1

        else:  # modo = "2"
            if i + 1 < len(oraciones):
                bloques.append(oraciones[i] + " " + oraciones[i+1])
                i += 2
            else:
                bloques.append(oraciones[i])
                i += 1

    # -------- Filtrar párrafos con demasiados números --------
    def demasiado_numeros(texto):
        nums = sum(c.isdigit() for c in texto)
        return nums / max(len(texto), 1) > 0.15 or bool(re.search(r"\b\d+(\.\d+)*\b", texto))

    bloques = [b for b in bloques if not demasiado_numeros(b)]

    # -------- Balancear preguntas --------
    preg = [b for b in bloques if "¿" in b or "?" in b]
    nopreg = [b for b in bloques if "¿" not in b and "?" not in b]

    # eliminar 75% de nopreg
    cant = int(len(nopreg) * 0.25)
    nopreg_reducidas = list(np.random.choice(nopreg, cant, replace=False)) if cant > 0 else []

    # combinar todo
    bloques_final = preg + nopreg_reducidas

    # mezclar aleatoriamente
    np.random.shuffle(bloques_final)

    df = pd.DataFrame(bloques_final, columns=['parrafo'])
    df.to_csv('libro_parrafos.csv', index=False, encoding='utf-8-sig')

    return df



In [15]:
def crearDataSet(libro,i,pdf=False):
  if pdf:
    convertir_pdf_a_csv2(libro)
  else:
    convertir_epub_a_csv(libro)

  df = pd.read_csv('libro_parrafos.csv')

  parrafos = pd.DataFrame(columns=['default', 'limpio'])
  parrafos['limpio'] = df['parrafo'].apply(normalize_text_X)
  parrafos['default'] = df['parrafo'].apply(normalize_text_y)

  columnas = ['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización']
  datos = pd.DataFrame(columns=columnas)

  for p in parrafos['default']:
    etiquetas = transformar_etiqueta([p], i)
    etiquetas = pd.DataFrame(etiquetas, columns=columnas)
    datos = pd.concat([datos, etiquetas], ignore_index=True)
    i += 1

  print('Data set creado de tamaño: ', datos.shape)

  numeric_cols = ['instancia_id', 'token_id', 'punt_inicial', 'punt_final', 'capitalización']
  for col in numeric_cols:
    datos[col] = pd.to_numeric(datos[col], errors='coerce')

  return parrafos, datos, i

In [ ]:
def agregar_embeddings(df_X):
  token_ids = df_X['token_id'].tolist()
  embeddings_list = []
  for token_id  in token_ids:
      if token_id is None or token_id == tokenizer.unk_token_id:
        token_id = tokenizer.unk_token_id
      # Detach the tensor before converting to list
      embedding = model.embeddings.word_embeddings.weight[token_id].detach().tolist()
      embeddings_list.append(embedding)
  df_X['embeddings'] = embeddings_list
  # The embeddings are already lists of floats, no need to convert to int
  # df_X['embeddings'] = df_X['embeddings'].apply(lambda x: [int(i) for i in x])

  return df_X

In [ ]:
def trasformar_df_dfPyTorch(datos_X, datos_Y):
  # Convert the list of lists in the 'embeddings' column to a NumPy array of floats
  embeddings_array = np.array(datos_X['embeddings'].tolist(), dtype=np.float32)
  X = torch.tensor(embeddings_array, dtype=torch.float32)
  Y = torch.tensor(datos_Y[['punt_inicial', 'punt_final', 'capitalización']].values, dtype=torch.float32)
  dataSetPT = TensorDataset(X, Y)

  return dataSetPT

## Cargar libros

Busca libro en formato epub de mi github

In [ ]:
path = 'https://raw.githubusercontent.com/AzulBarr/Aprendizaje-Automatico/main/TPs/tp2'

In [ ]:
libro1 = '/Harry_Potter_y_el_caliz_de_fuego_J_K_Rowling.epub'

In [ ]:
path = path + libro1

In [ ]:
!wget -O libro1.epub $path

##Codigo

In [ ]:
parrafos, dataSet = crearDataSet('libro1.epub')

In [ ]:
datos_X = dataSet[['instancia_id', 'token_id', 'token']]
datos_Y = dataSet[['punt_inicial', 'punt_final', 'capitalización']]

In [ ]:
datos_X_ext = agregar_embeddings(datos_X)

### Puntuacion inicial:
*   "": 0
*   "¿": 1

### Puntuacion final:
*   "": 0
*   ".": 1
*   ",": 2
*   "?": 3



In [ ]:
dataSetPT = trasformar_df_dfPyTorch(datos_X, datos_Y)

In [ ]:
X0, Y0 = dataSetPT[0]
print("X[0]:", X0)
print("Y[0]:", Y0)

CARGAR LIBROS PDF

In [ ]:
carpeta = "libros_pdf"
todos_parrafos = pd.DataFrame(columns=['default', 'limpio'])
todos_datos = pd.DataFrame(columns=['instancia_id', 'token_id', 'token', 'punt_inicial', 'punt_final', 'capitalización'])
i = 0
l = 0
for nombre in os.listdir(carpeta):
    ruta = os.path.join(carpeta, nombre)
    if nombre.lower().endswith(".pdf"):
        parrafos, datos, i = crearDataSet(ruta,i, pdf=True)
    elif nombre.lower().endswith(".epub"):
        parrafos, datos = crearDataSet(ruta, pdf=False)
    else:
        print(f"⚠️ Saltando {nombre} (no es PDF ni EPUB)")
        continue
    todos_parrafos = pd.concat([todos_parrafos, parrafos], ignore_index=True)
    todos_datos = pd.concat([todos_datos, datos], ignore_index=True)
    print(f"✅ Procesado el libro {l}: {nombre} ({len(parrafos)} párrafos, {len(datos)} tokens)")
    print(f"Cantidad de parrafos totales: {len(todos_parrafos)}")
    print(f"Cantidad de tokens totales: {len(todos_datos)}")
    l += 1
    todos_parrafos.to_csv("/content/drive/MyDrive/colab/dataset_parrafos_total3.csv", index=False)
    todos_datos.to_csv("/content/drive/MyDrive/colab/dataset_datos_total3.csv", index=False)
    print(f"Se guardó automáticamente en Drive")

print(f"\n💾 Dataset total creado: {len(todos_parrafos)} párrafos y {len(todos_datos)} tokens.")
